In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
# df=pd.read_csv('_chat.txt',sep='@#@#',header=None) #chat
df=pd.read_csv('_chat_flo.txt',sep='@#@#',header=None) #chat

len(df)

C:\Users\BNI\AppData\Local\Temp\ipykernel_1048\2914589366.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df=pd.read_csv('_chat_flo.txt',sep='@#@#',header=None) #chat


3799

In [3]:
df

,0
0,[05/08/21 15.08.36] halodek nightingle: thanks...
1,[05/08/21 15.10.05] atwin: yoo
2,[01/09/21 18.04.04] atwin: iya flo dari homecr...
3,[01/09/21 18.10.19] halodek nightingle: Gils 😂
4,[12/09/21 17.06.43] atwin: flooo
...,...
3794,[27/03/23 15.30.57] halodek nightingle: Ini be...
3795,[28/03/23 06.51.22] atwin: bedaa
3796,[28/03/23 06.51.24] atwin: gym nga
3797,[28/03/23 07.59.45] halodek nightingle: Baru b...


multiple line merged to the initial line as one record
(harder than i thought as >2 liners would be a problem, mandi duls ah)

In [4]:
add=''
ct=0
length=len(df[0])
df[0]=df[0].str.replace('\u200e','')
for i,row in enumerate(reversed(df[0])):
    if len(row)>=19 and (row[0] == '[' and row[18] == ']'and row[3]=='/' and row[12]=='.'):
        df[0][length-1-i]+=add
        add=''
    else:
        add+='\n'+row
        ct+=1
        df[0][length-1-i]='|was addition|'
        print(i,length-1-i,row)
      
ct

5 3793 Selamat untuk bp/ibu yg mutasi/promosi, semoga amanah dan sukses ..amin YRA..🙏🙏🙏
6 3792 12. Meiliana/PM SME sebagai WPW 2 W02
7 3791 11. Mesah Roni Ginting/DGM RTL sebagai GM RTL
8 3790 10. Sri Indira/GM RTL sebagai GM CRP
9 3789 9. Teddy Wishadi/GM CRP sebagai Direktur BNI Multifinance
10 3788 8. Sandy Dwinanto/DGM HUK sebagai GM HUK
11 3787 7. Johansyah/GM HUK sebagai GM RRC
12 3786 6. Bani Iqbal/GM RRC sebagai WPW 2 W18
13 3785 5. Yenni Sari Dewi/DGM HCS sebagai GM HCS
14 3784 4. Dandy P. Sjamsudin/GM HCS sebagai GM BCV
15 3783 3. Yogi Bima Sakti/GM SCS sebagai GM ENB
16 3782 2. Rangga Bhirawa Wicaksana/GM ENB sebagai GM CMB
17 3781 1. Martinus Matondang/GM CMB sebagai SCO CMB
728 3070 Be there DMA’ers, because it’s time to celebrate!💃
758 3040 ROWS 6 PRECEDING) AS 6MSum
759 3039 ORDER BY year, week
930 2868 - OVO & ShopeePay (gaada GOPAY) Ama: 082290806127
931 2867 - BNI Ama: 0390380223
932 2866 transfer boleh ke:
1078 2720 Aamiin Aamiin YRA 🤲
1079 2719 dan Margie beserta ke

108

In [5]:
print(len(df[df[0]=='|was addition|']),'messages was deleted to be merged with the nearest initial message')
df=df[df[0]!='|was addition|']

108 messages was deleted to be merged with the nearest initial message


ambil komponen

In [6]:
start,end = '] ',': '
df['sender']=df[0].apply(lambda x: x[x.find(start)+len(start):x.find(end)])
df['date']=df[0].str[1:19].str.strip().str.strip('[').str.strip(']').str[:8]
df['date']=pd.to_datetime(df['date'], dayfirst=True)
df['time']=df[0].str[1:19].str.strip().str.strip('[').str.strip(']').str[9:]
df['time']=pd.to_datetime(df['time'],format= '%H.%M.%S' ).dt.time
df['message']=df[0].apply(lambda x: x[x.find(': ')+len(': '):])
df

C:\Users\BNI\AppData\Local\Temp\ipykernel_1048\4120937632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sender']=df[0].apply(lambda x: x[x.find(start)+len(start):x.find(end)])
C:\Users\BNI\AppData\Local\Temp\ipykernel_1048\4120937632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date']=df[0].str[1:19].str.strip().str.strip('[').str.strip(']').str[:8]
C:\Users\BNI\AppData\Local\Temp\ipykernel_1048\4120937632.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,0,sender,date,time,message
0,[05/08/21 15.08.36] halodek nightingle: thanks...,halodek nightingle,2021-08-05,15:08:36,thanks win👍
1,[05/08/21 15.10.05] atwin: yoo,atwin,2021-08-05,15:10:05,yoo
2,[01/09/21 18.04.04] atwin: iya flo dari homecr...,atwin,2021-09-01,18:04:04,iya flo dari homecredit wkkwkw
3,[01/09/21 18.10.19] halodek nightingle: Gils 😂,halodek nightingle,2021-09-01,18:10:19,Gils 😂
4,[12/09/21 17.06.43] atwin: flooo,atwin,2021-09-12,17:06:43,flooo
...,...,...,...,...,...
3794,[27/03/23 15.30.57] halodek nightingle: Ini be...,halodek nightingle,2023-03-27,15:30:57,Ini beda mei kan??😅😅
3795,[28/03/23 06.51.22] atwin: bedaa,atwin,2023-03-28,06:51:22,bedaa
3796,[28/03/23 06.51.24] atwin: gym nga,atwin,2023-03-28,06:51:24,gym nga
3797,[28/03/23 07.59.45] halodek nightingle: Baru b...,halodek nightingle,2023-03-28,07:59:45,Baru bgn gue


cek validasi sender

In [ ]:
df[(df['sender']!='kameg') & (df['sender']!='atwin')]

In [ ]:
df[(df['date']>'2022-10-07') & (df['date']<'2022-10-09')].head(50)

export main

In [ ]:
# df.to_csv('main.csv',index=False)
df.to_excel('main.xlsx',index=False)

In [ ]:
df2=df.copy()
df['out']=1
df2['out']=0
time=df.append(df2)[['out','time']].reset_index(drop=1)
time.to_excel('time.xlsx',index=False)
time

words freq (wordcloud)

In [ ]:
for _ in df.message:
    

In [ ]:
df['sender'][10]

#### reply time

curating new dataframe that only contains first message of each reply 'batch' from a sender

In [ ]:
df=df.reset_index(drop=True)

df_first_text=pd.DataFrame()
# lst_ft=[]
for i,row in df.iterrows():
    if i > 0:
        if row['sender'] == df['sender'][i-1]:
            continue
#         print(df.iloc[1],row)
        df_first_text=df_first_text.append(row)
#         df_first_text=pd.concat([df_first_text,df.iloc[i]],axis=0) #outer, axis 0

time difference between replies 'batch'

In [ ]:
df_first_text['datetime']=pd.to_datetime(df_first_text['date'].astype(str) + ' ' + df_first_text['time'].astype(str))
df_first_text['time diff']=df_first_text['datetime'].diff()

filtering only messages where a conversation is replied within 1 day (reasonable duration for a message to be replied in an engaging conversation)

In [ ]:
# df_first_text['time diff'].time()#astype(str)
df_first_text['reply time (m)']=df_first_text['time diff'].dt.total_seconds()/60
print('before:',len(df_first_text))
#filter yang dibales lebih dari sehari
df_first_text=df_first_text[df_first_text['reply time (m)']<1440]
print('after:',len(df_first_text))

output

In [ ]:
df_first_text.to_excel('reply_time.xlsx',index=False)

In [ ]:
cek
# start=23
# df[df[0].str[start:start+9]=='gabriella']
# df[(df[0].str[8:9]=='m') ]
# df[(df[0].str[14]=='M') & (df[0].str[0]=='[')]